In [1]:
from surrogate.neighbor import GeneticAlgorithmPermutationGenerator, calculate_feature_values
from utils.prediction import PredictionTypeWrapper, PredictionTypeWrapper, generate_local_predictions, wrap_information
from surrogate.wrapper import SurrogateWrapper
from lore.prepare_dataset import prepare_diabetes_dataset

from utils import load

import tensorflow as tf
import numpy as np
import pandas as pd

### Set random seed
seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
### Load data
dataset = load.SelectableDataset.Diabetes
data, balanced_data, X, Y, encoder, scaler, n_features, n_classes, feature_names, target_name = load.load_dataset(dataset)
X_train, Y_train, X_test, Y_test, X_validation, Y_validation = load.load_training_data(dataset)
model = load.load_trained_model_for_dataset(dataset)

Loaded model from disk


In [3]:
diabetes_feature_range = (X_train.min(axis=0), X_train.max(axis=0))
# store all information 
local_data_dict = generate_local_predictions( X_test, Y_test, model, scaler, encoder )
# sorting by different conditions
true_positives,true_negatives, false_positives, false_negatives = wrap_information( local_data_dict )
# get all prediction types
all_predictions = PredictionTypeWrapper(true_positives,true_negatives, false_positives, false_negatives)

In [4]:
surrogate_wrapper = SurrogateWrapper(model)
diabetes_dataset = prepare_diabetes_dataset('./datasets/diabetes.csv')

In [5]:
diabetes_dataset['continuous']

['Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [6]:
diabetes_dataset['discrete']

['Pregnancies', 'Outcome']

In [7]:
X2E = X_test
y2E = surrogate_wrapper.predict(X2E).flatten()
y2E = np.asarray([diabetes_dataset['possible_outcomes'][i] for i in y2E])

In [8]:
diabetes_dataset['feature_values'] = calculate_feature_values(balanced_data, list(balanced_data.columns), 'Outcome', ['Pregnancies'], size=1000)

In [9]:
idx_features = diabetes_dataset['idx_features']
discrete = diabetes_dataset['discrete']
continuous = diabetes_dataset['continuous']
target_name = diabetes_dataset['class_name']
feature_values = diabetes_dataset['feature_values']

In [10]:
generator = GeneticAlgorithmPermutationGenerator(idx_features, discrete, continuous, target_name, surrogate_wrapper)

In [11]:
x = X2E[0,:]

In [12]:
generator.continuous

['Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [13]:
generator.generate_data(x, feature_values)

/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


56,
   0.0,
   0.4485842026825634,
   0.7874310665252113,
   20.356274539164687],
  [0.05882352941176471,
   0.9497487437185927,
   76.96187863270848,
   0.23232323232323235,
   28.11384201998729,
   32.69254408703011,
   0.6814007733545656,
   30.567067579981142],
  [0.05882352941176471,
   0.9497487437185927,
   0.49180327868852464,
   -2.8699414942536556,
   1.0,
   0.4485842026825634,
   0.5591571560646234,
   21.647503740857953],
  [0.05882352941176471,
   0.9497487437185927,
   0.49180327868852464,
   0.23232323232323235,
   28.11384201998729,
   0.4485842026825634,
   -0.04237937786942292,
   21.345723003338733],
  [0.05882352941176471,
   0.9497487437185927,
   0.49180327868852464,
   0.23232323232323235,
   28.11384201998729,
   0.4485842026825634,
   0.5591571560646234,
   21.647503740857953],
  [0.05882352941176471,
   0.9497487437185927,
   0.49180327868852464,
   0.23232323232323235,
   28.11384201998729,
   0.4485842026825634,
   0.5591571560646234,
   21.647503740857953]